# 6 Visualise labelled data and remove artifacts

Code that let us overlay each frame of video with outputs from the models. And create time series plots. 

In [3]:
import os
import cv2
import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd
import ultralytics.utils as ultrautils
import utils
import display
import json
import time

In [4]:
videos_in = os.path.join("..","LookitLaughter.test")
demo_data = os.path.join("..","data", "demo")
temp_out = os.path.join("..","data","0_temp")
data_out = os.path.join("..","data","1_interim")
videos_out = os.path.join("..","data","2_final")

metadata_file = "_LookitLaughter.xlsx"

In [ ]:
processedvideos = utils.getprocessedvideos(data_out)
processedvideos.head()

## 6.1 Add annotations to all vidoes.

Generate annotated videos for all videos in the test set.

In [ ]:
forceAnnotation = True

for index, r in processedvideos.iterrows():

    videopath = os.path.join(videos_in,r["VideoID"])
    videoname = os.path.basename(r["VideoID"])
    try: 
        #let's get all the annotations for this video
        kpts = utils.getKeyPoints(processedvideos,videoname)
        facedata = utils.getFaceData(processedvideos,videoname)
        speechdata = utils.getSpeechData(processedvideos,videoname)
    except FileNotFoundError as e:
        print(f"Data error for {videoname}\n" + "Error: " + str(e))
        continue
    if forceAnnotation or pd.isnull(r["annotatedVideo"]) or not os.path.exists(r["annotatedVideo"]):
        print(f"Creating annotated video for {videoname}")
        annotatedVideo = display.createAnnotatedVideo(videopath, kpts, facedata, speechdata, temp_out, False)
        vidwithaudio = display.addSoundtoVideo(annotatedVideo, r["Audio.file"], videos_out)
        r["annotatedVideo"] = vidwithaudio
        r["annotated.when"] = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime())
        #update this row in processedvideos dataframe
        processedvideos.loc[index] = r
    else:
        print(f"Already processed {r['VideoID']}")

#save the processedvideos dataframe
utils.saveprocessedvideos(processedvideos, data_out)

# 6.2 TODO Correct person labels in all videos.

Swap parent and child if these are wrong. Ignore other people in video.